In [55]:
import pandas as pd 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
class DatasetLoader:
    def __init__(self):
        self.vectorizer = TfidfVectorizer(encoding='utf-8', stop_words=stopwords.words('german'), ngram_range=(1,1), max_df=0.5, min_df=0, binary=True, norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=True)
       
    def load_train_data(self,csv_file_path):
        data_frame = pd.read_csv(csv_file_path)
        data = np.array(data_frame)
        self.train_ids = data[:,0]
        self.train_labels = np.asarray(data[:,1:3]).astype('float32')
        self.train_processed_tweets = self.vectorizer.fit_transform(data[:,3])
    def load_test_data(self,csv_file_path):
        data_frame = pd.read_csv(csv_file_path)
        data = np.array(data_frame)
        self.test_ids = data[:,0]
        self.test_processed_tweets = self.vectorizer.transform(data[:,1])
        
    def load_validation_data(self, csv_file_path):
        data_frame = pd.read_csv(csv_file_path)
        data = np.array(data_frame)
        self.validation_ids = data[:,0]
        self.validation_labels = np.asarray(data[:,1:3]).astype('float32')
        self.validation_processed_tweets = self.vectorizer.transform(data[:,3])
    
    

In [56]:
loader = DatasetLoader();
loader.load_train_data('training.csv')
print(loader.train_processed_tweets.shape)
print(loader.train_labels.shape)

loader.load_validation_data('validation.csv')
print(loader.validation_processed_tweets.shape)
print(loader.validation_labels.shape)
loader.load_test_data('test.csv')



(22583, 100014)
(22583, 2)
(3044, 100014)
(3044, 2)


In [53]:
# from sklearn.linear_model.stochastic_gradient import SGDClassifier
# from sklearn.multioutput import RegressorChain

# clf = SGDClassifier(loss='log', penalty='elasticnet', alpha=5e-5, l1_ratio=0.9, fit_intercept=True, max_iter=5, n_jobs=2, random_state=0, learning_rate="optimal")
# clfwrapper = RegressorChain(clf)
# clfwrapper.fit(loader.train_processed_tweets,loader.train_labels)

In [54]:
from sklearn.metrics import mean_squared_error
import csv 
class DataSetEvaluator:
    def calculateMSE(self, model, labels, processed_features):
        y_pred = model.predict(processed_features)
        print(y_pred[0])
        return mean_squared_error(labels, y_pred)
    def generateEvaluationFile(self,model,ids, processed_features, filename):
        y_pred = model.predict(processed_features)
        print(y_pred[0])
        file=open(filename,"w")
        writes=csv.writer(file,delimiter=',',quoting=csv.QUOTE_ALL)
        count =0
        for row in y_pred:
            writes.writerow([ids[int(count)],row[0],row[1]])
            count = count + 1
        
    

In [29]:
from sklearn.linear_model import LinearRegression
LinearRegressionModel = LinearRegression()
LinearRegressionModel.fit(loader.train_processed_tweets,loader.train_labels)

LinearRegression()

In [50]:
evaluator = DataSetEvaluator()
evaluator.calculateMSE(LinearRegressionModel, loader.validation_labels, loader.validation_processed_tweets)

[51.40825723  9.92682199]


0.8374783279600073

In [35]:
from sklearn.multioutput import RegressorChain
from sklearn.svm import LinearSVR

LinearSvrModel = LinearSVR()
# define the chained multioutput wrapper model
LinearSvrWrapper = RegressorChain(LinearSvrModel)
# fit the model on the whole dataset
LinearSvrWrapper.fit(loader.train_processed_tweets,loader.train_labels)

C:\Users\georg\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


RegressorChain(base_estimator=LinearSVR())

In [52]:
print(evaluator.calculateMSE(LinearSvrWrapper, loader.validation_labels, loader.validation_processed_tweets))
# evaluator.generateEvaluationFile(LinearSvrWrapper, loader.test_ids, loader.test_processed_tweets, 'linearSvrSubmission.txt')

[51.763421    9.60847543]
0.6197157054653172


In [36]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

modelNN = Sequential()
modelNN.add(Dense(512, input_shape=(max_words,), activation='relu'))
modelNN.add(Dropout(0.5))
modelNN.add(Dense(256, activation='sigmoid'))
modelNN.add(Dropout(0.5))
modelNN.add(Dense(2))
modelNN.summary()
modelNN.compile(loss='mean_squared_error', optimizer='adam')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               256512    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 514       
Total params: 388,354
Trainable params: 388,354
Non-trainable params: 0
_________________________________________________________________


In [37]:
modelNN.fit(loader.train_processed_tweets,loader.train_labels,
  batch_size=32,
  epochs=10,
  verbose=1,
  validation_split=0.1,
  shuffle=True)

ValueError: `validation_split` is only supported for Tensors or NumPy arrays, found following types in the input: [<class 'scipy.sparse.csr.csr_matrix'>]

In [20]:
evaluator = DataSetEvaluator()
evaluator.generateEvaluationFile(BayesModelWrapper, test_loadet, 'bayes-submission.txt')

[51.70891315  9.51027828]
